<h2>Mục Lục<span class="tocSkip"></span></h2>
<div class="toc"><ul class="toc-item"><li><span><a href="#Khám-phá-tập-dữ-liệu-Pima" data-toc-modified-id="Khám-phá-tập-dữ-liệu-Pima"><span class="toc-item-num">1&nbsp;&nbsp;</span>Thu thập dữ liệu</a></span></li><li><span><a href="#Tiền-xử-lý" data-toc-modified-id="Tiền-xử-lý"><span class="toc-item-num">2&nbsp;&nbsp;</span>Khám phá dữ liệu</a></span><ul class="toc-item"></ul></li><li><span><a href="#Mô-hình-Logistic-Regression" data-toc-modified-id="Mô-hình-Logistic-Regression"><span class="toc-item-num">3&nbsp;&nbsp;</span>Các câu hỏi cần trả lời</a></span></li><li><span><a href="#Trực-quan-dữ-liệu" data-toc-modified-id="Trực-quan-dữ-liệu"><span class="toc-item-num">4&nbsp;&nbsp;</span>Nhìn lại quá trình làm đồ án</a></span></li><li><span><a href="#Đánh-giá-mô-hình" data-toc-modified-id="Đánh-giá-mô-hình"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tài liệu tham khảo</a></span></li></ul></div>

# 1. Thu thập dữ liệu

- **Chủ đề của dữ liệu:** **`Traffic accident in UK`**
- **Nguồn dữ liệu:** **`Kaggle`**
- **Link:** https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales
- **License:** **`Open Database`**
- **Cách tác giả thu thập dữ liệu:** 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose

# 2. Khám phá dữ liệu

### Đọc dữ liệu từ file CSV đã tải xuống sẵn vào DataFrame

<b>Dữ liệu về số vụ tai nạn tại nước Anh từ năm 2012 - 2014</b>

In [2]:
df = pd.read_csv("accidents_2012_to_2014.csv")
df.head()

/home/luatvy/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
0,201201BS70001,527200,178760,-0.169101,51.493429,1,3,2,1,19/01/2012,...,Pedestrian phase at traffic signal junction,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002821,2012
1,201201BS70002,524930,181430,-0.200838,51.517931,1,3,2,1,04/01/2012,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01004760,2012
2,201201BS70003,525860,178080,-0.188636,51.487618,1,3,2,1,10/01/2012,...,non-junction pedestrian crossing,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002893,2012
3,201201BS70004,524980,181030,-0.200259,51.514325,1,3,1,1,18/01/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002886,2012
4,201201BS70005,526170,179200,-0.183773,51.497614,1,3,1,1,17/01/2012,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002890,2012


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [3]:
df.shape

(464697, 33)

Dữ liệu gồm 464.697 dòng và 33 cột

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả dữ liệu và quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin của một vụ tai nạn. Và các dòng thì có ý nghĩa giống nhau.

### Dữ liệu có các dòng bị lặp không?

In [4]:
df.duplicated().any()

True

Tồn tại những dòng trùng lặp. Và theo dự đoán rằng người thu thập dữ liệu này khi thu thập nhiều dữ liệu nhỏ để ghép thành một dữ liệu lớn này đã vô tình ghép trùng những dòng đã tồn tại nên ta sẽ xóa đi những dòng trùng lặp đồng thời reset index của df.

In [5]:
df = df.drop_duplicates().reset_index(drop=True)
df.shape

(430550, 33)

Như vậy dữ liệu còn 430.550 dòng

### Mỗi cột có ý nghĩa gì?

Vì người thu thập dữ liệu không nói rõ về các cột nên sẽ tìm một số nguồn khác có thông tin về các dữ liệu:
https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=7752&type=Data%20catalogue#!/documentation

Mô tả các cột như sau:
- **Accident_Index**: Mã định danh cho một vụ tai nạn.
- **Location_Easting_OSGR** và **Location_Northing_OSGR**: Tọa độ sử dụng Grid reference system để tham chiếu tìm vị trí. (Có thể sử dụng trang https://magic.defra.gov.uk/ để tìm vị trí của tọa độ đã cho) .
- **Longitude** và **Latitude**: Kinh độ và vĩ độ của vị trí xảy ra tai nạn.
- **Police_Force**: Lực lượng cảnh sát chịu trách nhiệm tại khu vực xảy ra tai nạn hoặc là lực lượng cảnh sát được giao nhiệm vụ xử lí vụ tai nạn???
- **Accident_Severity**: Mức độ của tai nạn (nhẹ, nặng,...) .
- **Number_of_Vehicles**: Số lượng phương tiện trong vụ tai nạn.
- **Number_of_Casualties**: Số lượng thương vong. (bị thương hoặc tử vong)
- **Date**: Ngày tháng năm xảy ra tai nạn
- **Day_of_Week**: Thứ trong tuần xảy ra tai nạn. (chủ nhật, thứ hai, ...)
- **Time**: Giờ và phút xảy ra tai nạn.
- **Local_Authority_(District)**: thị trấn, quận nơi xảy ra tai nạn.
- **Local_Authority_(Highway)**: Cơ quan chịu trách nhiệm vận hành, quản lý và bảo trì đường giao thông công cộng tại nơi xảy ra tai nạn.
- **1st_Road_Class**: Phân loại đường. (đường của nhà nước, đường của địa phương tự quản, ...)
- **1st_Road_Number**: Tên đường xảy ra tai nạn. (dạng số)
- **Road_Type**: Loại đường (đường một chiều, đường nhánh, ...)
- **Speed_limit**: Tốc độ cho phép trên con đường xảy ra tai nạn.
- **Junction_Detail**: Chi tiết nút giao thông.
- **Junction_Control**: kiểm soát nút giao thông. (đèn tín hiệu tự động, tự do di chuyển không kiểm soát, ...).
- **2nd_Road_Class**: Nếu vụ tai nạn xảy ra tại giao hai con đường thì là phân loại cho đường thứ hai.
- **2nd_Road_Number**: Tương tự trên nếu xảy ra tại nơi giao hai con đường thì là số đường của đường thứ hai.
- **Pedestrian_Crossing-Human_Control**: Có hay không người điều khiển khu vực qua đường dành cho người đi bộ trong bán kính 50 mét ở khu tai nạn.
- **Pedestrian_Crossing-Physical_Facilities**: Cơ sở vật chất của khu vực qua đường cho người đi bộ (vạch kẻ đường, đường hầm, cầu vượt, ...)
- **Light_Conditions**: Điều kiện ánh sáng tại nơi xảy ra tai nạn.
- **Weather_Conditions**: Điều kiện thời tiết tại nơi xảy ra tai nạn.
- **Road_Surface_Conditions**: Điều kiện mặt đường tại nơi xảy ra tai nạn.
- **Special_Conditions_at_Site**: Điều kiện đặc biệt nào khác tại nơi xảy ra tai nạn. (đường hư, ...)
- **Carriageway_Hazards**: Có vật cản gì tại nơi xảy ra tai nạn hay không. (Động vật băng qua đường, ...)
- **Urban_or_Rural_Area**: Nội ô hay ngoại ô.
- **Did_Police_Officer_Attend_Scene_of_Accident**: Cảnh sát có đến hiện trường vụ tai nạn không.
- **LSOA_of_Accident_Location**: Vị trí cho những vùng nhỏ không có trên bản đồ lớn ở Anh.
- **Year**: Năm xảy ra tai nạn

Các dữ liệu về số, mã của vùng được tra ở file `7752_road-accident-safety-data-guide.xls`

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [6]:
df.dtypes

Accident_Index                                  object
Location_Easting_OSGR                            int64
Location_Northing_OSGR                           int64
Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Date                                            object
Day_of_Week                                      int64
Time                                            object
Local_Authority_(District)                       int64
Local_Authority_(Highway)                       object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                       object
Speed_limit                                      int64
Junction_D

**Các cột có dạng kiểu object**

In [7]:
obj_cols = df.select_dtypes(include='object').columns
obj_cols

Index(['Accident_Index', 'Date', 'Time', 'Local_Authority_(Highway)',
       'Road_Type', 'Junction_Control', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location'],
      dtype='object')

Ta sẽ xem cụ thể trong các cột Object có các kiểu dữ liệu gì?

In [8]:
def open_object_dtype(s):
    dtypes = set()
    s.apply(lambda x: dtypes.add(type(x)))
    return dtypes

In [9]:
for i in range(0,len(obj_cols)):    
    print(obj_cols[i], open_object_dtype(df[obj_cols[i]]))

Accident_Index {<class 'str'>}
Date {<class 'str'>}
Time {<class 'str'>, <class 'float'>}
Local_Authority_(Highway) {<class 'str'>}
Road_Type {<class 'str'>}
Junction_Control {<class 'str'>, <class 'float'>}
Pedestrian_Crossing-Human_Control {<class 'str'>}
Pedestrian_Crossing-Physical_Facilities {<class 'str'>}
Light_Conditions {<class 'str'>}
Weather_Conditions {<class 'str'>}
Road_Surface_Conditions {<class 'str'>, <class 'float'>}
Special_Conditions_at_Site {<class 'str'>, <class 'float'>}
Carriageway_Hazards {<class 'str'>, <class 'float'>}
Did_Police_Officer_Attend_Scene_of_Accident {<class 'str'>, <class 'float'>}
LSOA_of_Accident_Location {<class 'str'>, <class 'float'>}


* Ta sẽ tiền xử lí ở cột `Date` khi dữ liệu ở đây chưa ở dạng datetime để dễ xử lí.
* Còn các cột chứa chuỗi str và chứa cả kiểu float tức là chứa các giá trị nan.

## Tiền xử lí

### Chuyển dtype của cột "Date" sang datetime

In [10]:
df.Date = pd.to_datetime(df.Date)
df.Time = pd.to_datetime(df.Time).dt.time

## Tiếp tục khám phá dữ liệu

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

**Theo như quan sát thì có các cột**: "Location_Easting_OSGR", "Location_Northing_OSGR", "Longitude", "Latitude", "Police_Force", "Accident_Severity", "Number_of_Vehicles", "Number_of_Casualties", "Date", "Day_of_Week", "Time", "Local_Authority_(District)", "1st_Road_Class", "1st_Road_Number", "2nd_Road_Class", "2nd_Road_Number", "Urban_or_Rural_Area", "Year" có vẻ là ở dạng numeric. <br>
**Nhưng**, "Police_Force", "Accident_Severity", "Local_Authority_(District)", "1st_Road_Class", "2nd_Road_Class", "Urban_or_Rural_Area" là các cột dạng categorical (vì là chuỗi hoặc mã đã được đánh dưới dạng số).<br>
**Như vậy ta có 12 cột Numeric**: "Location_Easting_OSGR",
 "Location_Northing_OSGR",
 "Longitude",
 "Latitude",
 "Number_of_Vehicles",
 "Number_of_Casualties",
 "Date",
 "Day_of_Week",
 "Time",
 "1st_Road_Number",
 "2nd_Road_Number",
 "Year"

Ta sẽ tìm tỉ lệ % các giá trị thiếu, min và max của mỗi cột numeric.

In [11]:
df_temp=df[["Location_Easting_OSGR", "Location_Northing_OSGR", "Longitude", "Latitude", "Number_of_Vehicles", "Number_of_Casualties", "Date", "Day_of_Week", "Time", "1st_Road_Number", "2nd_Road_Number", "Year"]]
def missing_ratio(df):
    return df.isna().mean() * 100
def Min(df):
    return df.min()
def Max(df):
    return df.max()
nume_col_profiles_df = df_temp.agg([missing_ratio,min,max])
nume_col_profiles_df

,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,1st_Road_Number,2nd_Road_Number,Year
missing_ratio,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.003019,0.0,0.0,0.0
min,65510.0,10290.0,-7.509162,49.912941,1.0,1.0,2012-01-01 00:00:00,1.0,NaN,0.0,-1.0,2012.0
max,655370.0,1190858.0,1.759382,60.597984,67.0,93.0,2014-12-31 00:00:00,7.0,NaN,9999.0,9999.0,2014.0


Cột Time có giá trị thiếu các cột còn lại thì đầy đủ. Ở cột 2nd_Road_Number giá trị min là -1, theo mô tả thì -1 tức là nan (không có giá trị).<br>
Số lượng giá trị thiếu của Time là rất rất nhỏ so với tổng số lượng dòng nên ta sẽ bỏ đi các giá trị thiếu này luôn.

In [12]:
df=df.dropna(subset=['Time'])

In [13]:
nume_col_profiles_df = df_temp.agg([missing_ratio,min,max])
nume_col_profiles_df

,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,1st_Road_Number,2nd_Road_Number,Year
missing_ratio,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.003019,0.0,0.0,0.0
min,65510.0,10290.0,-7.509162,49.912941,1.0,1.0,2012-01-01 00:00:00,1.0,NaN,0.0,-1.0,2012.0
max,655370.0,1190858.0,1.759382,60.597984,67.0,93.0,2014-12-31 00:00:00,7.0,NaN,9999.0,9999.0,2014.0


### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Có 33-12 = 21 cột Categorical gồm: "Accident_Index",
 "Police_Force",
 "Accident_Severity",
 "Local_Authority_(District)",
 "Local_Authority_(Highway)",
 "1st_Road_Class",
 "Road_Type",
 "Speed_limit",
 "Junction_Detail",
 "Junction_Control",
 "2nd_Road_Class",
 "Pedestrian_Crossing-Human_Control",
 "Pedestrian_Crossing-Physical_Facilities",
 "Light_Conditions",
 "Weather_Conditions",
 "Road_Surface_Conditions",
 "Special_Conditions_at_Site",
 "Carriageway_Hazards",
 "Urban_or_Rural_Area",
 "Did_Police_Officer_Attend_Scene_of_Accident",
 "LSOA_of_Accident_Location"

Với mỗi cột categorical, ta tính tỉ lệ % giá trị thiếu, số lượng giá trị khác nhau, list/array các giá trị khác nhau.

In [14]:
df_temp = df[["Accident_Index", "Police_Force", "Accident_Severity", "Local_Authority_(District)", "Local_Authority_(Highway)", "1st_Road_Class", "Road_Type", "Speed_limit", "Junction_Detail", "Junction_Control", "2nd_Road_Class", "Pedestrian_Crossing-Human_Control", "Pedestrian_Crossing-Physical_Facilities", "Light_Conditions", "Weather_Conditions", "Road_Surface_Conditions", "Special_Conditions_at_Site", "Carriageway_Hazards", "Urban_or_Rural_Area", "Did_Police_Officer_Attend_Scene_of_Accident", "LSOA_of_Accident_Location"]]
def num_diff_vals(df):
    return len(df.value_counts().index)
def diff_vals(df):
    return list(df.unique())
cate_col_profiles_df = df_temp.agg([missing_ratio,num_diff_vals,diff_vals])
cate_col_profiles_df

,Accident_Index,Police_Force,Accident_Severity,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,Junction_Control,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
missing_ratio,0,0,0,0,0,0,0,0,100,38.927,...,0,0,0,0,0.172343,0.000464536,0.000696804,0,0.000464536,6.66702
num_diff_vals,263811,51,3,380,207,6,6,7,0,4,...,3,6,5,9,5,8,6,2,2,34284
diff_vals,"[201201BS70001, 201201BS70002, 201201BS70003, ...","[1, 48, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 16,...","[3, 2, 1]","[12, 1, 11, 28, 570, 3, 5, 4, 8, 2, 9, 30, 31,...","[E09000020, E09000033, E09000013, E09000005, E...","[3, 4, 5, 6, 1, 2]","[Single carriageway, One way street, Roundabou...","[30, 50, 40, 20, 70, 60, 10]",[nan],"[Automatic traffic signal, Giveway or uncontro...",...,"[None within 50 metres, Control by other autho...","[Pedestrian phase at traffic signal junction, ...","[Darkness: Street lights present and lit, Dayl...","[Fine without high winds, Raining without high...","[Dry, Wet/Damp, Frost/Ice, Snow, Flood (Over 3...","[None, Roadworks, Mud, Auto traffic singal out...","[None, Other object in carriageway, Any animal...","[1, 2]","[Yes, No, nan]","[E01002821, E01004760, E01002893, E01002886, E..."


Ở các cột categorical thì có vẻ giá trị thiếu đã nhiều hơn so với các cột Numeric.

# 3. Các câu hỏi cần trả lời

# 4. Nhìn lại quá trình làm đồ án

# 5. Tài liệu tham khảo